# Project

## Download Data

In [ ]:
import urllib
import json
import pandas as pd
from bd_addons.interpro_data import *

url_general = "https://www.ebi.ac.uk/interpro/api/protein/entry/pfam/pf03060?format=json"
url_reviewed = "https://www.ebi.ac.uk/interpro/api/protein/reviewed/entry/pfam/pf03060?format=json"
url_unreviewed = "https://www.ebi.ac.uk/interpro/api/protein/unreviewed/entry/pfam/pf03060?format=json"

metadata_reviewed, entries_reviewed, metadata_unreviewed, entries_unreviewed = get_data(url_reviewed, url_unreviewed, url_general, 1)

Remaining: 23 proteins
Got reviewed data
Remaining: 62885 proteins
Remaining: 62865 proteins
Remaining: 62845 proteins
Remaining: 62825 proteins
Remaining: 62805 proteins
Remaining: 62785 proteins
Remaining: 62765 proteins
Remaining: 62745 proteins
Remaining: 62725 proteins
Remaining: 62705 proteins
Remaining: 62685 proteins
Remaining: 62665 proteins
Remaining: 62645 proteins
Remaining: 62625 proteins
Remaining: 62605 proteins
Remaining: 62585 proteins
Remaining: 62565 proteins
Remaining: 62545 proteins
Remaining: 62525 proteins
Remaining: 62505 proteins
Remaining: 62485 proteins
Remaining: 62465 proteins
Remaining: 62445 proteins
Remaining: 62425 proteins
Remaining: 62405 proteins
Remaining: 62385 proteins
Remaining: 62365 proteins
Remaining: 62345 proteins
Remaining: 62325 proteins
Remaining: 62305 proteins
Remaining: 62285 proteins
Remaining: 62265 proteins
Remaining: 62245 proteins
Remaining: 62225 proteins
Remaining: 62205 proteins
Remaining: 62185 proteins
Remaining: 62165 protei

In [ ]:
metadata_ureviewed.head(5)

In [ ]:
entries_unreviewed.head(5)